# DEMO: CLOE

**Description**: this DEMO allows the user to compute the theoretical predictions of the galaxy clustering and weak gravitational lensing probes along with the likelihood given some benchmark data. This notebook uses `CLOE` along with `Cobaya`.

It contains three potential different scenarios for the user (referred to as user-cases) corresponding to CASE 1, 2 and 3 in the notebook. A basic description of each of the user-cases is the following:

* **CASE 1**: samples the posterior distribution of the parameters of interest (cosmological and nuisance parameters) by running `Cobaya` with the external Euclid Likelihood of `CLOE` and the sampler of choice from `Cobaya`. At the moment, this user-case executes a single point evaluation of the likelihood using the `evaluate` sampler of `Cobaya`.

* **CASE 2**: creates a `model` of `Cobaya` using an internal wrapper of `Cobaya` itself. This wrapper is based on the `evaluate` sampler of `Cobaya`. The `model` allows the user to make a single computation of the priors, likelihood, and posterior, measure the time needed by each module or retrieve the information of which theoretical quantities were asked to the Boltzman solvers, for instance. This `model` instance is important, because it is the way the connection between `Cobaya` and the `EuclidLikelihood` code is made internally within `cobaya_interface.py`. In fact, the `model` instance is essential if you want to run CASE 2.1.

* **CASE 2.1**: the final section of the notebook retrieves and plots internal theoretical quantities computed by the `CLOE` code. Be aware that, to be able to retrieve and plot quantities, you need to first run **CASE 2**, because CASE 2.1 requires the `model` instance to be loaded. You can plot the following quantities:
    * $H(z)$: Hubble parameter
    * $r(z)$, $D_A(z)$: comoving and angular diameter distances
    * $\sigma_8$, $f\sigma_8$: amplitude of the (linear) matter power spectrum on scales of 8 $h^{-1}$ Mpc and its product with the growth rate
    * $D(z)$, $f(z)$: growth factor and growth rate
    * $P_{\delta\delta}, P_{gg}, P_{g\delta}, P_{\delta i}, P_{gi}, P_{ii}$: matter power spectrum obtained by the Boltzman solver, and retrieved from `Cobaya`, and different power spectra
    * $n(z)$: galaxy density distributions
    * $W_i^{\rm GC}(z), W_i^{\rm \gamma}(z), W_i^{\rm IA}(z), W_i^{\rm RSD}(z), W_i^{\rm mag}(z)$ : window functions for galaxy clustering (GC), shear ($\gamma$), intrinsic alignments (IA), redshift space distortions (RSD), and magnification bias (mag).
    * $C_\ell$: angular power spectra
    * $P_\ell$: Legendre multipole power spectra


**README**: https://gitlab.euclid-sgs.uk/pf-ist-likelihood/likelihood-implementation/-/blob/master/README.md

**Install**: in order to use this DEMO notebook, you need to clone the repository `https://gitlab.euclid-sgs.uk/pf-ist-likelihood/likelihood-implementation.git`, and install CLOE as described in the README.
Alternatively, you may be ready to run if ```Cobaya``` and ```CAMB/CLASS``` are installed. See details below. 

**Cobaya documentation**: https://cobaya.readthedocs.io/en/latest/

**Python information**: if the user is not comfortable with some Python vocabulary used in this notebook,
a nice python review can be found here: https://wiki.python.org/moin/BeginnersGuide

In [ ]:
# General Python imports

import numpy as np
import matplotlib.pyplot as plt
import time
import os, sys

In [ ]:
# If you have cloned the repository and opened this notebook,
# this notebook should be in likelihood-implementation/notebooks
# Let's set the working directory to be likelihood-implementation

likelihood_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
sys.path.insert(0, likelihood_path)
print('Setting as working directory: ', likelihood_path)

In [ ]:
# Matplotlib params setup

%matplotlib inline
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('font',size=25)
plt.rc('axes', titlesize=26)
plt.rc('axes', labelsize=25)
plt.rc('lines', linewidth=2)
plt.rc('lines', markersize=6)
plt.rc('legend', fontsize=20)
plt.rc('mathtext', fontset='stix')
plt.rc('font', family='STIXGeneral')

`Cobaya` needs some modules to run: i.e: `CAMB`, `CLASS`, `Polychord`, other likelihood codes, and data (Planck 2018, DES, ...)
If you have installed `CLOE` as in the README instructions, you need to activate the conda environment `cloe` to run this notebook. This conda environment has CAMB/CLASS/Nonlinear codes installed, so you do not need to worry about anything else.

Alternatively, if you have already installed Cobaya and other cosmological codes such as CAMB, CLASS, Polychord, or Planck 2018, you have two options. Be aware that we **discourage** to go with these two options at the moment:

* **(1) Only for CAMB/CLASS in the `theory` block, Polychord in the `mcmc` block and `plik` in the `likelihood` block**:  point out where each of them is installed with the flag 'path' in the dictionary of Cell 5. See the comment in Cell 5 corresponding to the 'theory' key of the 'info' dictionary. If CLOE was not installed with conda, be aware that the Bacco emulator `baccoemu` and the Euclid emulator `euclidemu2` will need to be installed globally or locally by `pip install the_emulator (--user)` where `the_emulator` is `baccoemu`/`euclidemu2`. The flag `--user` makes pip install the package locally and it may be needed when runnning in a computer/cluster with no superuser permissions.
 

* **(2) Only if you have CAMB/CLASS/Polychord installed automatically by Cobaya**: if you installed the cosmological codes as Cobaya automatic installation suggests  (https://cobaya.readthedocs.io/en/latest/installation_cosmo.html) you need to point out the path to your modules as in the variable `packages_path` below. If CLOE was not installed with conda, please be aware that you will need to have  `baccoemu` and `euclidemu2` installed globally or locally to use these emulators. See (1) for more information. 

In [ ]:
# ATTENTION: write down the path to your COBAYA modules if you want to follow option (2) above.
# Otherwise skip this cell
# packages_path = "YOUR_PATH_HERE"

In [ ]:
## ATTENTION: turn to True if you would like to save all plots in a repository called figs
if not os.path.exists('figs/'):
    os.mkdir('figs/')
save_plots = True

## CASE 1: Run `Cobaya` importing CLOE as an external likelihood
**User-case**: *Run and go*. This is the most straightforward case where the user calls `Cobaya` to sample the posterior distribution of the parameters of interest. At the moment, this user-case runs one computation of the likelihood at one point in parameter space given the corresponding theoretical predictions of the Euclid observables.

To run, Cobaya needs an 'input file'. Please read the comments in the cells below carefully to understand how this input file looks like and which options are available to be modified by the user.

In [ ]:
# Import external loglike from CLOE; that is, the class EuclidLikelihood within cobaya_interface.py

from cloe.cobaya_interface import EuclidLikelihood

In [ ]:
# Let us consider EuclidLikelihood as an external likelihood class for Cobaya.
# Cobaya needs a dictionary or yaml file as input to start running.
# This dictionary below ('info') can be modified by the user according to their needs.
# The options that the user can modify are highlighted with the acronym (UC).

info = {
    #'params': Cobaya-protected key
    # Includes the parameters that the user wants to sample over.
'params': {
        # (UC): each parameter below (which is a 'key' of another sub-dictionary)
        # can contain a dictionary with the key 'prior', 'latex', ...
        # If the prior dictionary is not passed to a parameter, this parameter is fixed.
        # In this example, we are sampling the spectral index parameter ns.
        # For more information, see Cobaya's quickstart example: https://cobaya.readthedocs.io/en/latest/example.html
        'ombh2': 0.022445, #Reduced baryon density parameter
        'omch2': 0.1205579307, #Reduced CDM density parameter
        'H0': 67.0, #Hubble constant in km/s/Mpc
        'tau': 0.0925, #Optical depth
        'mnu': 0.06, #Sum of neutrino masses in eV
        'nnu': 3.046, #Effective number of neutrinos
        'As': 2.12605e-9, #Amplitude of the primordial scalar power spectrum
        'ns': {'prior':{'min':0.8, 'max':1.2}}, #Spectral index (uniform prior)
        'w': -1.0, #Present-day dark energy equation of state in CPL model
        'wa': 0.0, #Derivative of dark energy equation of state in CPL model
        'omk': 0.0, #Curvature density parameter
        'omegam': None, #Derived parameter: Matter density parameter
        'omegab': None, #Derived parameter: Baryon density parameter
        'omeganu': None, #Derived parameter: Neutrino density parameter
        'omnuh2': None, #Derived parameter: Reduced neutrino density parameter
        'omegac': None, #Derived parameter: CDM density parameter
        'N_eff': None},
    #'theory': Cobaya-protected key
    # Cobaya needs to ask some minimum theoretical requirements to a Boltzman solver
    # (UC): Choose between CAMB or CLASS for the Boltzmann solver.
    # Here, we use CAMB and specify some CAMB arguments, such as
    # the number of massive neutrinos and DE model.
    #
    # ATTENTION: If you have CAMB/CLASS already installed and
    # you are not using the likelihood conda environment
    # or option (2) in cell (3) (Cobaya modules), you can add an extra key called 'path' within the camb dictionary
    # to point to your already installed CAMB code
    # NOTE: for values of the nonlinear flag larger than 0, a new key is added in info['theory']['camb']['extra_args'],
    # i.e. 'halofit_version', which contains the requested version of Halofit/HMCODE, as described above
    'theory': {'camb':
               {'stop_at_error': True,
                'extra_args':{'num_massive_neutrinos': 1,
                              'dark_energy_model': 'ppf'}}},
    #'sampler': Cobaya-protected key
    # (UC): Choose the sampler.
    # Check Cobaya's documentation to see the list of available samplers.
    # For a full MCMC sampling, use the following line:
    # 'sampler': {'mcmc': {'max_tries': 100000}},
    # Here, use the 'evaluate' sampler to make a single likelihood computation.
    'sampler': {'evaluate': None},
    # 'packages_path': Cobaya-protected key
    # This is the variable you need to update
    # if you are running Cobaya with cobaya_modules (option (2) above).
    # If you are using the conda likelihood environment or option (1),
    # please, keep the line below commented
    #
    #'packages_path': modules_path,
    #
    #'output': Cobaya-protected key
    # This is where the chains are stored.
    # (UC): modify the path below within 'output' to choose a name and a directory for those files
    'output': 'chains/my_euclid_experiment',
    #'debug': Cobaya-protected key
    # (UC): This specifies how much information to print, can be set to True/False.
    'debug': False,
    #'timing': Cobaya-protected key
    # (UC): if timing: True, Cobaya returns how much time it took it to make a computation of the likelihood
    # and how much time it takes each of the modules to perform their tasks
    'timing': True,
    #'force': Cobaya-protected key
    # (UC): if 'force': True, Cobaya forces deleting the previous output files, if found, with the same name
    'force': True,
    }

In [ ]:
#'likelihood': Cobaya-protected key
# (UC): The user can select which data to use for the analysis.
# Check Cobaya's documentation to see the list of the current available data experiments
# In this DEMO, we load the EuclidLikelihood as an external function, and name it 'Euclid'
info['likelihood'] = {'Euclid':
                     {'external': EuclidLikelihood, # Likelihood Class to be read as external
                     # Note: everything down below will overwrite the information read in the config folder
                     #
                     # Select which observables to use during the analysis by setting them to True or False
                     'observables_selection': {
                         'WL': {'WL': True, 'GCphot': True, 'GCspectro': False},
                         'GCphot': {'GCphot': True, 'GCspectro': False},
                         'GCspectro': {'GCspectro': True},
                         'CG': {'CG': False},
                         # Add RSD to photometric probes
                         'add_phot_RSD': False,
                         # Switch to allow for matrix transformations of theory and data vectors
                         'matrix_transform_phot' : False # 'BNT' or 'BNT-test'
                     },
                     'use_magnification_bias_spectro': 0,
                     'use_Weyl': False,
                     # Plot the selected observables matrix
                     'plot_observables_selection': True,
                      # Nonlinear flags
                      # NL_flag_phot_matter
                        # 0 -> linear-only
                        # 1 -> Halofit (Takahashi)
                        # 2 -> Mead2016 (includes baryon corrections)
                        # 3 -> Mead2020 (w/o baryon corrections)
                        # 4 -> Euclid Emulator 2
                        # 5 -> BACCO (matter)
                     'NL_flag_phot_matter': 0,
                      # NL_flag_spectro
                        # 0 -> linear-only
                        # 1 -> EFT
                     'NL_flag_spectro': 0,
                      # Baryonic feedback flag
                      # With this, the user can specify which baryon model they want
                      # For the time-being the available options are:
                            #0 -> no baryonic feedback
                            #1 -> Mead2016
                            #2 -> Mead2020_feedback
                            #3 -> BCemu
                            #4 -> BACCO (baryonic feedback)
                     'NL_flag_phot_baryon': 0,
                     # NL_flag_phot_bias
                        # 0 -> linear only
                        # 1 -> Non-linear PT
                     'NL_flag_phot_bias': 0,
                     # IA_flag
                        # 0 -> NLA
                        # 1 -> TATT
                     'IA_flag':1,
                     # IR-resummation method
                        # 'DST' -> Discrete sine transform
                        # 'EH' -> Eisenstein-Hu                      
                     'IR_resum': 'DST',
                     # Use magnification bias for GC spectro
                     'use_magnification_bias_spectro': 0,
                      # Baryonic feedback z_dependence flag selector for
                      # Bacco or BCemu emulators
                         # False -> 1 set of parameters per redshift bin
                         # True  -> Power law dependence on z
                     'Baryon_redshift_model': False,
                     'solver': 'camb',
                     'params': {
                                # (UC): galaxy bias parameters:
                                # The bias parameters below are currently fixed to the
                                # values used by the Inter Science Taskforce: Forecast (IST:F)
                                # and presented in the corresponding IST:F paper (arXiv: 1910.09273).
                                # However, they can be changed by the user and even sampled over by putting a prior.
                                # Photometric galaxy bias parameters
                                'b1_photo_bin1': 1.0997727037892875,
                                'b1_photo_bin2': 1.220245876862528,
                                'b1_photo_bin3': 1.2723993083933989,
                                'b1_photo_bin4': 1.316624471897739,
                                'b1_photo_bin5': 1.35812370570578,
                                'b1_photo_bin6': 1.3998214171814918,
                                'b1_photo_bin7': 1.4446452851824907,
                                'b1_photo_bin8': 1.4964959071110084,
                                'b1_photo_bin9': 1.5652475842498528,
                                'b1_photo_bin10': 1.7429859437184225,
                                # Magnification bias parameters
                                'magnification_bias_1': 0.0,
                                'magnification_bias_2': 0.0,
                                'magnification_bias_3': 0.0,
                                'magnification_bias_4': 0.0,
                                'magnification_bias_5': 0.0,
                                'magnification_bias_6': 0.0,
                                'magnification_bias_7': 0.0,
                                'magnification_bias_8': 0.0,
                                'magnification_bias_9': 0.0,
                                'magnification_bias_10': 0.0,
                                # Shear calibration multiplicative bias parameters
                                'multiplicative_bias_1': 0.0,
                                'multiplicative_bias_2': 0.0,
                                'multiplicative_bias_3': 0.0,
                                'multiplicative_bias_4': 0.0,
                                'multiplicative_bias_5': 0.0,
                                'multiplicative_bias_6': 0.0,
                                'multiplicative_bias_7': 0.0,
                                'multiplicative_bias_8': 0.0,
                                'multiplicative_bias_9': 0.0,
                                'multiplicative_bias_10': 0.0,
                                # Intrinsic alignment parameters
                                'a1_ia': 1.72,
                                'a2_ia': 2,
                                'b1_ia': 1,
                                'eta1_ia': -0.41,
                                'eta2_ia': 1,
                                'beta1_ia': 0.0,
                                # Redshift distribution nuisance parameters: shifts
                                'dz_1_GCphot': 0.0, 'dz_1_WL': 0.0,
                                'dz_2_GCphot': 0.0, 'dz_2_WL': 0.0,
                                'dz_3_GCphot': 0.0, 'dz_3_WL': 0.0,
                                'dz_4_GCphot': 0.0, 'dz_4_WL': 0.0,
                                'dz_5_GCphot': 0.0, 'dz_5_WL': 0.0,
                                'dz_6_GCphot': 0.0, 'dz_6_WL': 0.0,
                                'dz_7_GCphot': 0.0, 'dz_7_WL': 0.0,
                                'dz_8_GCphot': 0.0, 'dz_8_WL': 0.0,
                                'dz_9_GCphot': 0.0, 'dz_9_WL': 0.0,
                                'dz_10_GCphot': 0.0, 'dz_10_WL': 0.0,
                                'gamma_MG': 0.55,
                                'sigma_z': 0.002,
                                 # Spectroscopic galaxy bias parameters
                                'b1_spectro_bin1': 1.441,
                                'b1_spectro_bin2': 1.643,
                                'b1_spectro_bin3': 1.862,
                                'b1_spectro_bin4': 2.078,
                                #Spectroscopic magnification bias
                                'magnification_bias_spectro_bin1': 0.79,
                                'magnification_bias_spectro_bin2': 0.87,
                                'magnification_bias_spectro_bin3': 0.96,
                                'magnification_bias_spectro_bin4': 0.98,},
                     # k values for extrapolation of the matter power spectrum and k-array size
                     'k_max_extrap': 500.0,
                     'k_min_extrap': 1E-5,
                     'k_samp': 1000,
                     # z limit values and z-array size
                     'z_min': 0.0,
                     'z_max': 4.0,
                     'z_samp': 100,
                     # Use MG gamma
                     'use_gamma_MG': False,
                     # Use Weyl bypass
                     'use_Weyl':False,
                     # Use redshift-dependent purity for GCspectro or not
                     'f_out_z_dep': False,
                     # Add spectroscopic redshift errors
                     'GCsp_z_err' : True,
                     # Print theory predictions
                     'print_theory' : False,
                     #'data': This gives specifications for the paths of the input data files
                     'data': {
                        #'sample' specifies the first folder below the main data folder
                        'sample': 'ExternalBenchmark',
                        #'spectro' and 'photo' specify paths to data files.
                        'spectro': {
                            # GC Spectro root name should contain z{:s} string
                            # to enable iteration over bins
                            'root': 'cov_power_galaxies_dk0p004_z{:s}.fits',
                            'redshifts': ["1.", "1.2", "1.4", "1.65"],
                            'edges': [0.9, 1.1, 1.3, 1.5, 1.8],
                            'scale_cuts_fourier': 'GCspectro-FourierSpace.yaml',
                            'root_mixing_matrix': 'mm_FS230degCircle_m3_nosm_obsz_z0.9-1.1.fits',
                            'Fourier': True},
                        'photo': {
                          'photo_data': 'standard',
                            'redshifts': [0.2095, 0.489, 0.619, 0.7335, 0.8445, 0.9595, 1.087, 1.2395, 1.45, 2.038],
                            'ndens_GC': 'niTab-EP10-RB00.dat',
                            'ndens_WL': 'niTab-EP10-RB00.dat',
                            'luminosity_ratio': 'luminosity_ratio.dat',
                            # Photometric root names should contain z{:s} string
                            # to specify IA model
                            'root_GC': 'Cls_{:s}_PosPos.dat',
                            'root_WL': 'Cls_{:s}_ShearShear.dat',
                            'root_XC': 'Cls_{:s}_PosShear.dat',
                            'root_mixing_matrix': 'fs2_mms_10zbins_32ellbins.fits',
                            'IA_model': 'zNLA',
                            # Photometric covariances root names should contain z{:s} string
                            # to specify how the covariance was calculated
                            'cov_GC': 'CovMat-PosPos-{:s}-20Bins.npz',
                            'cov_WL': 'CovMat-ShearShear-{:s}-20Bins.npz',
                            'cov_3x2pt': 'CovMat-3x2pt-{:s}-20Bins.npz',
                            'cov_model': 'Gauss',  # or 'BNT-Gauss' if BNT selected above
                            'Fourier': True
                            }},
                    }}

### Create the yaml file corresponding to the Likelihood Parameters
Cobaya needs to know which parameters the likelihood `EuclidLikelihood` expects (for example, nuisance parameters), apart from the cosmological parameters (i.e: $\Omega_b$, $H_0$, $n_s$...) which are already known by the theory code (CAMB/CLASS). 

For that, a `yaml` file is generated instead of having those likelihood parameters hard-coded within `cobaya_interface.py`. To generate this `yaml` file, execute either the next cell or the next-to-next cell.

Please be aware that this function will only make Cobaya expect those likelihood parameters. To set some fixed values or put a prior on them, the user needs to do so within the `info` python dictionary above. 

In [ ]:
from cloe.auxiliary.likelihood_yaml_handler import set_halofit_version
# Set the correct halofit version under the 'extra_args' field in ['theory']['camb']
set_halofit_version(info, info['likelihood']['Euclid']['NL_flag_phot_matter'],
                          info['likelihood']['Euclid']['NL_flag_phot_baryon'])
# Now we are ready to run Cobaya.

In [ ]:
# An alternative way from using a hardcoded Cobaya info dictionary is loading the Cobaya configuration from a yaml file.
# Run this cell only if you want to load the info dictionary from file.
run_this_cell = False
# WARNING: running this cell will overload the info dictionary defined in the previous cells

from cloe.auxiliary.likelihood_yaml_handler import get_default_configs_path
from cloe.auxiliary.likelihood_yaml_handler import update_cobaya_params_from_model_yaml
from cloe.auxiliary.likelihood_yaml_handler import update_cobaya_dict_with_halofit_version
from cloe.auxiliary.yaml_handler import yaml_read

if run_this_cell:
    # A test configuration yaml file is in 'configs/config_test.yaml'
    # We can play with this file, or use another config file ...
    config_test_path = get_default_configs_path() / 'config_test.yaml'
    # Loading the dictionary from the file
    config_test_dict = yaml_read(config_test_path)
    info = config_test_dict['Cobaya']
    # And read the model file name (only for this case, 'params' value is a file name where the model is specified)
    # We can play with this model file, or use another model file ...
    model_file = info['params']
    # Joining the complete path for the model file
    model_path = config_test_path.parent.joinpath(model_file)
    # Updating the Cobaya info dictionary from the model
    # i.e. writing the whole 'params' dict instead of the model file name
    update_cobaya_params_from_model_yaml(info, model_path)
    # Updating the Cobaya dictionary to add the halofit version corresponding to the selected NL flag
    update_cobaya_dict_with_halofit_version(info)
    set_halofit_version(info, info['likelihood']['Euclid']['NL_flag_phot_matter'],
                              info['likelihood']['Euclid']['NL_flag_phot_baryon'])


# Now we are ready to run Cobaya.

In [ ]:
# Import Cobaya run function
from cobaya.run import run
#import matplotlib
#matplotlib.rcParams['text.usetex'] = False

# Let's run Cobaya
# the function run returns
# info_updated: an information dictionary updated with the defaults,
# equivalent to the updated yaml file produced by the shell invocation
# samples: a sampler object, with a sampler.products()
# being a dictionary of results.
# For the mcmc sampler, the dictionary contains only one chain under the key sample.
info_updated, samples = run(info)

In [ ]:
samples.products()

**Explanation of the printed output by Cobaya**: 
- output: it tells details about the output
- Prior: it shows the values sampled by the prior. In this case, our parameters of interest are all fixed so Cobaya reminds that.
- CAMB: it calls the theory code you wanted to use (CAMB/CLASS, in this case, CAMB, and where it is installed).

You see these outputs repeated twice because `CLOE` calls `Cobaya` internally twice, within the EuclidLikelihood, to calculate the fiducial cosmology.
- evaluate: the 'evaluate' sampler gets initialized, looks for a point and evaluates the posterior.

Note that since the option timing is True in the info dictionary, Cobaya tells you how much time it took to run CAMB and compute the likelihood (CLOE).

# Case 2: Run `Cobaya model` with CLOE as external likelihood 

**User-case**: the analysis tool `Cobaya` has a wrapper of its `evaluate` sampler (the one executed in CASE 1), called `model`, that allows us to:

* make an evaluation in a single point of prior, likelihood, and posterior distributions
* retrieve derived parameters and other quantities

The `model` wrapper is useful for debugging as well (i.e. imagine you are performing an MCMC sampling to find the best-fit values of a given model, and the MCMC chains get stuck for unknown reasons). With the `model`, you can investigate what is going on internally in `Cobaya` at each step of the sampling algorithm.

Moreover, our EuclidLikelihood code, which is designed to work as an external likelihood code for `Cobaya` relies on this `model` wrapper to make the connection between `Cobaya`, the likelihood calculation, and the computation of the theoretical predictions (see **CASE 2.1** for further information).

In this **CASE 2**, we will see how to activate the `model` wrapper, how to use it and how to understand the output it provides.

In [ ]:
# First: import model wrapper of Cobaya
from cobaya.model import get_model

# The `get_model` function of Cobaya imported in the line above needs a yaml or dictionary as an argument
# exactly the same as the function `run` in cell 9 also needs.
#
# We measure the time to give us an estimation of how long it takes to initialize the likelihood

t1 = time.time()

# Second: create an instance of the `model` wrapper called model
model = get_model(info)
print('Time for initialization of the likelihood: ', time.time()-t1)

**Explanation of the printed output by Cobaya**: 

- Model: it tells you Cobaya is using the `model` wrapper, and it's reading the info dictionary.
- Prior: it shows the values sampled by the prior. In this case, our parameters of interest are all fixed so Cobaya reminds that.
- CAMB: it calls the theory code you wanted to use (CAMB/CLASS, in this case, CAMB, and where it is installed).

You see these outputs repeated twice because CLOE calls Cobaya internally twice, within EuclidLikelihood, to calculate the fiducial cosmology.

It takes around 10 seconds to initialize the likelihood (reading OU-LE3 data and computing fiducial cosmology). Note this is only performed once in a Monte Carlo run. 

### Functionalities of the `model` wrapper:
As mentioned above, we can get an insight of what `Cobaya` is doing using the `model` object. We can ask, for instance:
 * (1) which quantities were **required** by the likelihood code and asked to the Boltzman solver through `Cobaya`
 * (2) at which values (i.e. redshift, scales...) those requirements were **requested**.
 
To see how to ask for these quantities, execute the cell below:

In [ ]:
# (1) Requirements needed by the likelihood code.
# That means, which quantities are we asking to the Boltzman (CAMB/CLASS) through Cobaya?
print('\n Requirements \n')
print(model.provider.requirement_providers)
# (2) At which values have the requirements been requested (redshift, scales...)?
print('\n Requested \n')
print(model.requested())

In [ ]:
#print(model.provider.requirement_providers)

* (3) With the `model` wrapper you can also make an evaluation of the prior, likelihood, and posterior distributions

ATTENTION: we initialized the `model` wrapper by reading the `info` dictionary above. This `info` dictionary has almost all the parameters of interest fixed (the only sampled parameters is $n_s$). Therefore, to make an evaluation of the probability distributions (priors, likelihoods, and posterior), you need:

* First: get a point of the sampled parameters from the prior distribution 
* Second: pass this point to the logposterior method

See comments in the cell below to understand how to retrieve the values of these distributions.

In [ ]:
# At the moment, we are sampling only ns
# If there are sampled parameters, we need first to obtain a value from the prior
# i.e: (FIRST)
point = dict(zip(model.parameterization.sampled_params(),
                 model.prior.sample(ignore_external=True)[0]))
t1 = time.time()
# (3) Make a computation of the logposterior on that point
logposterior = model.logposterior(point)
# If there were no sampled parameters, you can simply execute
#logposterior = model.logposterior({})
t2 = time.time()

# Note that we are measuring the time for illustration purposes only.

print('Time to compute the logposterior: ', t2-t1)
print('Full log-posterior:')
print('   logposterior: %g' % logposterior.logpost)
print('   logpriors: %r' % dict(zip(list(model.prior), logposterior.logpriors)))
print('   loglikelihoods: %r' % dict(zip(list(model.likelihood), logposterior.loglikes)))
print('   derived params: %r' % dict(zip(list(model.parameterization.derived_params()), logposterior.derived)))

# Case 2.1: EuclidLikelihood package

**User-case**: this case allows the user to go one level deeper down into the `CLOE` code, so that the user can retrieve and plot:

* Benchmark data used as mock data during the calculation of the likelihood. For instance:
    * $n(z)$: galaxy density distributions
* Cosmological quantities provided by the Boltzman solver through Cobaya. For example:
    * $H(z)$: Hubble factor
    * $r(z)$, $D_A$: comoving and angular diameter distances
    * $\sigma_8$, $f\sigma_8$: amplitude of the (linear) power spectrum on scales of 8 $h^{-1}$ Mpc and its product with the growth rate
* Internal cosmological quantities computed by the EuclidLikelihood package itself:
    * $D(z)$, $f(z)$: growth factor and growth rate
    * $P_{\delta\delta}, P_{gg}, P_{g\delta}, P_{\delta i}, P_{gi}, P_{ii}$: matter power spectrum obtained by the Boltzman solver in Cobaya and different power spectra in CLOE
* Theoretical predictions of the photometric and spectroscopic observables
    * $W_i^{\rm GC}(z), W_i^{\rm \gamma}(z), W_i^{\rm IA}(z), W_i^{\rm RSD}(z), W_i^{\rm mag}(z)$ : window functions or kernel for galaxy clustering (GC), shear ($\gamma$), intrinsic alignments (IA), redshift space distortions (RSD), and magnification bias (mag).
    * $C_\ell$: angular power spectra
    * $P_\ell$: Legendre multipole power spectra
* computation of the $\chi^2$

To be able to access all of these quantities and get a grasp of what EuclidLikelihood actually does, **you need to have loaded an instance of the `model` wrapper of Cobaya (explained in CASE 2)**. In reality, what we are doing in this **CASE 2.1** is to reproduce the steps that are done internally by `Cobaya` at each step of the sampling procedure within the file `cobaya_interface.py` of the likelihood package. Therefore, understanding **CASE 2.1** will also help the user to understand the details of the EuclidLikelihood source code and structure.



In [ ]:
# Create an instance of the class EuclidLikelihood
like = EuclidLikelihood()

# Initialize default parameters for redshift, k-array, fiducial cosmology, ...
like.initialize()

# Get the cosmo_dictionary where all the cosmology + theory parameters are saved
# ATTENTION: as explained above, you need to pass the `cobaya wrapper model` initialized
# in CASE 2 as an argument of the function, as well as the parameters of your theory.
# In CASE 1, when only Cobaya run is used, it internally creates this `model` instance itself
like.passing_requirements(model, info, **model.provider.params)

# Update the cosmology dictionary with interpolators + basic quantities such as
# P_gg, P_delta...
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], 0.05)

# Show what the cosmo_dic actually contains
print('\nKeys of the cosmo_dic: \n', list(like.cosmo.cosmo_dic.keys()))
print('\nKeys of the nuisance params within cosmo_dic: \n', list(like.cosmo.cosmo_dic['nuisance_parameters'].keys()))

In [ ]:
# You can also access the quantities of the Euclike module as follows:
# This function will return the loglike
loglike = like.likefinal.loglike(like.cosmo.cosmo_dic)

In [ ]:
# After the computation of the chi2, you can access the following quantities of the Euclike module
print('\nList of attributes of the euclike object: \n', list(vars(like.likefinal)))

In [ ]:
# You can print, for instance, the data read by CLOE
print(like.likefinal.data)

### Plot internal quantities and cosmological observables

In [ ]:
import seaborn as sns
sns.set_theme(style="ticks")
sns.set_palette(sns.color_palette("Paired"))

plt.rc('xtick',labelsize=20)
plt.rc('ytick',labelsize=20)
plt.rc('font',size=20)
plt.rc('axes', titlesize=25)
plt.rc('axes', labelsize=20)
plt.rc('lines', linewidth=3)
plt.rc('lines', markersize=6)
plt.rc('legend', fontsize=20)

In [ ]:
# Let's plot the background quantities of cosmo_dic
# See http://pf-ist-likelihood.pages.euclid-sgs.uk/likelihood-implementation/likelihood.cosmo.cosmology.html
# for extra information

fig, axs = plt.subplots(2, 2, figsize=(13,10))
for ax in axs.flatten():
    ax.tick_params(axis='both',which='both',direction='in')
fig.suptitle('Background quantities')
axs[0, 0].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['H_z_func'](like.cosmo.cosmo_dic['z_win']))
axs[0, 0].set_xlabel(r'$z$')
axs[0, 0].set_ylabel(r'$H(z)$ $[\rm km \times s^{-1} Mpc^{-1}]$')
axs[0, 1].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['d_z_func'](like.cosmo.cosmo_dic['z_win']),
              label = r'$D_{\rm{A}}(z)$ $[\rm{Mpc}]$')
axs[0, 1].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['r_z_func'](like.cosmo.cosmo_dic['z_win']), '--',
              label = r'$r(z)$')
axs[0, 1].set_xlabel(r'$z$')
axs[0, 1].legend(frameon=False)
axs[1, 0].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['sigma8_z_func'](like.cosmo.cosmo_dic['z_win']),
              label = r'$\sigma_8(z)$')
axs[1, 0].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['fsigma8_z_func'](like.cosmo.cosmo_dic['z_win']),
              '--', label = r'$f\sigma_8(z)$')
axs[1, 0].set_xlabel(r'$z$')
axs[1, 0].legend(frameon=False)
axs[1, 1].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['D_z_k'],
              label = r'$D(z)$')
axs[1, 1].plot(like.cosmo.cosmo_dic['z_win'],
               like.cosmo.cosmo_dic['f_z'](like.cosmo.cosmo_dic['z_win']),
              '--', label = r'$f(z) = f\sigma_8/\sigma_8$')
axs[1, 1].set_xlabel(r'$z$')
axs[1, 1].legend(frameon=False)

plt.tight_layout()
plt.subplots_adjust(top=0.90)
if save_plots:
    plt.savefig('figs/background_quantities.pdf')

In [ ]:
# Let's plot the matter power spectrum and other spectra quantities
# 'Pgg_spectro', 'Pgg_phot', 'Pgdelta_phot', 'Pgdelta_spectro', 'Pii', 'Pdeltai', 'Pgi_phot', 'Pgi_spectro'
# See http://pf-ist-likelihood.pages.euclid-sgs.uk/likelihood-implementation/likelihood.cosmo.cosmology.html
# for extra information

k_label           = r'$k$ $[\rm{Mpc}^{-1}]$'
pm_label          = r'$P_{\delta\delta} \left[\rm{Mpc}^3\right]$'
pgg_spectro_label = r'$P_{\rm{gg}}^{\,\rm{spectro}} \left[\rm{Mpc}^3\right]$'
pgg_phot_label    = r'$P_{\rm{gg}}^{\,\rm{phot}} \left[\rm{Mpc}^3\right]$'
pgd_spectro_label = r'$P_{\rm{g}\delta}^{\,\rm{spectro}} \left[\rm{Mpc}^3\right]$'
pgd_phot_label    = r'$P_{\rm{g}\delta}^{\,\rm{phot}} \left[\rm{Mpc}^3\right]$'
pII_label         = r'$P_{\,\rm{II}} \left[\rm{Mpc}^3\right]$'
pdI_label         = r'$P_{\delta\rm{I}} \left[\rm{Mpc}^3\right]$'
pgI_phot_label    = r'$P_{\rm{gI}}^{\,\rm{phot}} \left[\rm{Mpc}^3\right]$'
pgI_spectro_label = r'$P_{\rm{gI}}^{\,\rm{spectro}} \left[\rm{Mpc}^3\right]$'


import warnings
warnings.filterwarnings('ignore')
fig, axs = plt.subplots(9, 2, figsize=(13,47))
for ax in axs.flatten():
    ax.tick_params(axis='both',which='both',direction='in')
fig.suptitle('Spectra')
ks=np.logspace(-5, 0, 100)
axs[0, 0].loglog(ks, like.cosmo.cosmo_dic['Pk_delta'].P(like.cosmo.cosmo_dic['z_win'][0], ks), 'k-',
           label=r"$z = {:.2f}$".format(like.cosmo.cosmo_dic['z_win'][0]))
axs[0, 0].set_xlabel(k_label)
axs[0, 0].set_ylabel(pm_label)
axs[0, 0].legend(frameon=False)
#------
axs[0, 1].loglog(ks, like.cosmo.cosmo_dic['Pk_delta'].P(like.cosmo.cosmo_dic['z_win'][-1], ks), 'k-',
           label=r"$z = {:.2f}$".format(like.cosmo.cosmo_dic['z_win'][-1]))
axs[0, 1].set_xlabel(k_label)
axs[0, 1].set_ylabel(pm_label)
axs[0, 1].legend(frameon=False)
#------
for i, k in enumerate(ks):
    axs[1, 0].loglog(k, like.cosmo.cosmo_dic['Pgg_spectro'](
    like.cosmo.cosmo_dic['z_win'][25], k, 1), 'o', color=sns.color_palette("Paired")[0],
                     label=r"$z = {:.2f}$".format(
        like.cosmo.cosmo_dic['z_win'][25]) if i == 0 else "")
    axs[1, 1].loglog(k, like.cosmo.cosmo_dic['Pgg_spectro'](
    like.cosmo.cosmo_dic['z_win'][35], k, 1), 'o', color=sns.color_palette("Paired")[0],
                     label=r"$z = {:.2f}$".format(
        like.cosmo.cosmo_dic['z_win'][35]) if i == 0 else "")
axs[1, 0].set_xlabel(k_label)
axs[1, 0].set_ylabel(pgg_spectro_label)
axs[1, 1].set_xlabel(k_label)
axs[1, 1].set_ylabel(pgg_spectro_label)
axs[1, 0].legend(frameon=False)
axs[1, 1].legend(frameon=False)

#------
for i, k in enumerate(ks):
    axs[2, 0].loglog(k, like.cosmo.cosmo_dic['Pgg_phot'](
    like.cosmo.cosmo_dic['z_win'][0], k), 'o', color=sns.color_palette("Paired")[1],
                     label=r"$z = {:.2f}$".format(
        like.cosmo.cosmo_dic['z_win'][0]) if i == 0 else "")
    axs[2, 1].loglog(k, like.cosmo.cosmo_dic['Pgg_phot'](
    like.cosmo.cosmo_dic['z_win'][-1], k), 'o', color=sns.color_palette("Paired")[1],
                     label=r"$z = {:.2f}$".format(
        like.cosmo.cosmo_dic['z_win'][-1]) if i == 0 else "")
axs[2, 0].set_xlabel(k_label)
axs[2, 0].set_ylabel(pgg_phot_label)
axs[2, 1].set_xlabel(k_label)
axs[2, 1].set_ylabel(pgg_phot_label)
axs[2, 0].legend(frameon=False)
axs[2, 1].legend(frameon=False)

#------
for i, k in enumerate(ks):
    axs[3, 0].loglog(k, like.cosmo.cosmo_dic['Pgdelta_spectro'](
    like.cosmo.cosmo_dic['z_win'][25], k, 1), 'o', color=sns.color_palette("Paired")[2],
                     label=r"$z = {:.2f}$".format(
        like.cosmo.cosmo_dic['z_win'][25]) if i == 0 else "")
    axs[3, 1].loglog(k, like.cosmo.cosmo_dic['Pgdelta_spectro'](
    like.cosmo.cosmo_dic['z_win'][35], k, 1), 'o', color=sns.color_palette("Paired")[2],
                     label=r"$z = {:.2f}$".format(
        like.cosmo.cosmo_dic['z_win'][35]) if i == 0 else "")
axs[3, 0].set_xlabel(k_label)
axs[3, 0].set_ylabel(pgd_spectro_label)
axs[3, 1].set_xlabel(k_label)
axs[3, 1].set_ylabel(pgd_spectro_label)
axs[3, 0].legend(frameon=False)
axs[3, 1].legend(frameon=False)


#------
for i, k in enumerate(ks):
    axs[4, 0].loglog(k, like.cosmo.cosmo_dic['Pgdelta_phot'](
    like.cosmo.cosmo_dic['z_win'][0], k), 'o', color=sns.color_palette("Paired")[3],
                     label=r"$z = {:.2f}$".format(
        like.cosmo.cosmo_dic['z_win'][0]) if i == 0 else "")
    axs[4, 1].loglog(k, like.cosmo.cosmo_dic['Pgdelta_phot'](
    like.cosmo.cosmo_dic['z_win'][-1], k), 'o', color=sns.color_palette("Paired")[3],
                     label=r"$z = {:.2f}$".format(
        like.cosmo.cosmo_dic['z_win'][-1]) if i == 0 else "")
axs[4, 0].set_xlabel(k_label)
axs[4, 0].set_ylabel(pgd_phot_label)
axs[4, 1].set_xlabel(k_label)
axs[4, 1].set_ylabel(pgd_phot_label)
axs[4, 0].legend(frameon=False)
axs[4, 1].legend(frameon=False)

#------
for i, k in enumerate(ks):
    axs[5, 0].loglog(k, like.cosmo.cosmo_dic['Pii'](
    like.cosmo.cosmo_dic['z_win'][0], k), 'o', color=sns.color_palette("Paired")[4],
                     label=r"$z = {:.2f}$".format(
        like.cosmo.cosmo_dic['z_win'][0]) if i == 0 else "")
    axs[5, 1].loglog(k, like.cosmo.cosmo_dic['Pii'](
    like.cosmo.cosmo_dic['z_win'][-1], k), 'o', color=sns.color_palette("Paired")[4],
                     label=r"$z = {:.2f}$".format(
        like.cosmo.cosmo_dic['z_win'][-1]) if i == 0 else "")
axs[5, 0].set_xlabel(k_label)
axs[5, 0].set_ylabel(pII_label)
axs[5, 1].set_xlabel(k_label)
axs[5, 1].set_ylabel(pII_label)
axs[5, 0].legend(frameon=False)
axs[5, 1].legend(frameon=False)

#------
for i, k in enumerate(ks):
    axs[6, 0].semilogx(k, like.cosmo.cosmo_dic['Pdeltai'](
    like.cosmo.cosmo_dic['z_win'][0], k), 'o', color=sns.color_palette("Paired")[5],
                       label=r"$z = {:.2f}$".format(
        like.cosmo.cosmo_dic['z_win'][0]) if i == 0 else "")
    axs[6, 1].semilogx(k, like.cosmo.cosmo_dic['Pdeltai'](
    like.cosmo.cosmo_dic['z_win'][-1], k), 'o', color=sns.color_palette("Paired")[5],
                       label=r"$z = {:.2f}$".format(
        like.cosmo.cosmo_dic['z_win'][-1]) if i == 0 else "")
axs[6, 0].set_xlabel(k_label)
axs[6, 0].set_ylabel(pdI_label)
axs[6, 1].set_xlabel(k_label)
axs[6, 1].set_ylabel(pdI_label)
axs[6, 0].legend(frameon=False, loc=3)
axs[6, 1].legend(frameon=False, loc=3)

#------
for i, k in enumerate(ks):
    axs[7, 0].semilogx(k, like.cosmo.cosmo_dic['Pgi_phot'](
    like.cosmo.cosmo_dic['z_win'][0], k), 'o', color=sns.color_palette("Paired")[6],
                       label=r"$z = {:.2f}$".format(
        like.cosmo.cosmo_dic['z_win'][0]) if i == 0 else "")
    axs[7, 1].semilogx(k, like.cosmo.cosmo_dic['Pgi_phot'](
    like.cosmo.cosmo_dic['z_win'][-1], k), 'o', color=sns.color_palette("Paired")[6],
                       label=r"$z = {:.2f}$".format(
        like.cosmo.cosmo_dic['z_win'][-1]) if i == 0 else "")
axs[7, 0].set_xlabel(k_label)
axs[7, 0].set_ylabel(pgI_phot_label)
axs[7, 1].set_xlabel(k_label)
axs[7, 1].set_ylabel(pgI_phot_label)
axs[7, 0].legend(frameon=False, loc=3)
axs[7, 1].legend(frameon=False, loc=3)

#------
for i, k in enumerate(ks):
    axs[8, 0].semilogx(k, like.cosmo.cosmo_dic['Pgi_spectro'](
    like.cosmo.cosmo_dic['z_win'][0], k), 'o', color=sns.color_palette("Paired")[7],
                       label=r"$z = {:.2f}$".format(
        like.cosmo.cosmo_dic['z_win'][0]) if i == 0 else "")
    axs[8, 1].semilogx(k, like.cosmo.cosmo_dic['Pgi_spectro'](
    like.cosmo.cosmo_dic['z_win'][-1], k), 'o', color=sns.color_palette("Paired")[7],
                       label=r"$z = {:.2f}$".format(
        like.cosmo.cosmo_dic['z_win'][-1]) if i == 0 else "")
axs[8, 0].set_xlabel(k_label)
axs[8, 0].set_ylabel(pgI_spectro_label)
axs[8, 1].set_xlabel(k_label)
axs[8, 1].set_ylabel(pgI_spectro_label)
axs[8, 0].legend(frameon=False, loc=3)
axs[8, 1].legend(frameon=False, loc=3)

plt.tight_layout()
plt.subplots_adjust(top=0.97)
if save_plots:
    plt.savefig('figs/power_spectra.pdf')

In [ ]:
# We can also plot the initial data read by the like_calc module (class Euclike).
# This data is at the moment the fiducial data stored within the data folder of the repository

# i.e: let's plot the galaxy distributions for GCphot and WL (at the moment, they are the same)
zs = np.linspace(0, 4, 1000)
fig, axs = plt.subplots(2, 1, figsize=(10,10))
i = 0
j = 0
for ax in axs.flatten():
    ax.tick_params(axis='both',which='both',direction='in')
for key, value in like.likefinal.data_ins.nz_dict_WL.items():
    axs[0].plot(zs, value(zs), label = '$n_{{{}}}$'.format(i+1), linewidth=2, color=sns.color_palette("mako", 10)[i])
    i=i+1
for key, value in like.likefinal.data_ins.nz_dict_GC_Phot.items():
    axs[1].plot(zs, value(zs), label = '$n_{{{}}}$'.format(j+1), linewidth=2, color=sns.color_palette("rocket", 10)[j])
    j=j+1
axs[0].set_xlabel(r'$z$')
axs[0].set_ylabel(r'$n(z)$ $\left[\rm{sr}^{-1}\right]$')
axs[0].set_title(r'$\rm WL$')
axs[0].legend(frameon=False, ncol=2)
axs[1].set_xlabel(r'$z$')
axs[1].set_ylabel(r'$n(z)$ $\left[\rm{sr}^{-1}\right]$')
axs[1].set_title(r'$\rm GCphot$')
axs[1].legend(frameon=False, ncol=2);
plt.tight_layout()
if save_plots:
    plt.savefig('figs/n_z.pdf')

In [ ]:
# We can also plot the window functions for the photometric observables
# For that, you need to import the photo class and read the cosmology dictionary and the n(z) distributions above
from cloe.photometric_survey.photo import Photo
photo = Photo(like.cosmo.cosmo_dic, like.likefinal.data_ins.nz_dict_WL, like.likefinal.data_ins.nz_dict_GC_Phot, None)

In [ ]:
import matplotlib.colors as mcolors
color = list(mcolors.TABLEAU_COLORS.values())
fig, axs = plt.subplots(5, 1, figsize=(13,20))
for ax in axs.flatten():
    ax.tick_params(axis='both',which='both',direction='in')
for i in range(0, 10):
    axs[0].plot(zs, photo.WL_window(zs, i+1), '-', linewidth=2, color=sns.color_palette("mako", 10)[i], label='$n_{{{}}}$'.format(i+1))
    axs[1].plot(zs, photo.IA_window(zs, i+1), '-', linewidth=2, color=sns.color_palette("mako", 10)[i], label='$n_{{{}}}$'.format(i+1))
    axs[2].plot(zs, photo.GC_window(zs, i+1), '-', linewidth=2, color=sns.color_palette("rocket", 10)[i], label='$n_{{{}}}$'.format(i+1))
    axs[3].plot(zs, photo.magnification_window(zs, i+1), '-', linewidth=2, color=sns.color_palette("rocket", 10)[i], label='$n_{{{}}}$'.format(i+1))
    axs[4].plot(zs[3:], photo.GC_window_RSD(zs[3:], 40, i+1)[0][0], '-', linewidth=2, color=sns.color_palette("rocket", 10)[i], label='$n_{{{}}}$'.format(i+1))

axs[0].set_xlabel(r'$z$')
axs[0].set_ylabel(r'$W_i^{\gamma}$ $\left[\rm{Mpc}^{-1}\right]$')
axs[0].legend(frameon=False, ncol=2)
axs[1].set_xlabel(r'$z$')
axs[1].set_ylabel(r'$W_i^{\rm{IA}}$ $\left[\rm{Mpc}^{-1}\right]$')
axs[1].legend(frameon=False, ncol=2)
axs[2].set_xlabel(r'$z$')
axs[2].set_ylabel(r'$W_i^{\rm GCphot}$ $\left[\rm{Mpc}^{-1}\right]$')
axs[2].legend(frameon=False, ncol=2)
axs[3].set_xlabel(r'$z$')
axs[3].set_ylabel(r'$W_i^{\rm mag}$ $\left[\rm{Mpc}^{-1}\right]$')
axs[3].legend(frameon=False, ncol=2)
axs[4].set_xlabel(r'$z$')
axs[4].set_ylabel(r'$W_i^{\rm RSD}$ $\left[\rm{Mpc}^{-1}\right]$')
axs[4].legend(frameon=False, ncol=2)
plt.tight_layout()
if save_plots:
    plt.savefig('figs/window_functions.pdf')

### Plot Euclid final observables

We can use the auxiliary module of the CLOE to plot the final observables. **Have in mind that the predicted values for the angular power spectra will probably not agree with the fiducial plotted values from the Benchmark data as we are currently sampling $n_s$.**

In [ ]:
# We import the Plotter Class
from cloe.auxiliary.plotter import Plotter

In [ ]:
# Print if there is any sampled parameter
for k in model.parameterization.sampled_params():
    print(k)

In [ ]:
# Plotting the photometric observables for these redshift bins
bin_i = 1
bin_j = 1

In [ ]:
# WL angular power spectrum
fig1 = plt.figure()
ax1 = fig1.add_subplot(1, 1, 1)
ax1.tick_params(axis='both',which='both',direction='in')
for param in model.parameterization.sampled_params().keys():
        pl_label = 'ns'#'{}={:.4f}'.format(param, model.parameterization.sampled_params()[param])
        print('WARNING: param {} is being sampled! Possible mismatch between benchmark and theoretical prediction'.format(param))
pl_label = '$n_s = {:.2f}$'.format(model.parameterization.sampled_params()['ns'])
pl_inst = Plotter(cosmo_dic=like.cosmo.cosmo_dic, data=like.likefinal.data)
ax1 = pl_inst.plot_external_Cl_phot(bin_i, bin_j, ax1, probe='WL', pl_label='$\\rm Benchmark$')
ax1 = pl_inst.plot_Cl_phot(np.logspace(1, 3.6, 10), bin_i, bin_j, ax1, probe='WL', pl_colour='r', pl_linestyle='--', pl_label=pl_label)
ax1.set_xlabel(r'$\ell$', fontsize=20)
ax1.set_ylabel('$C_{1,1}^{\\rm{WL}}(\\ell) \\left[\\rm{sr}\\right]$')
ax1.set_xscale('linear')
ax1.set_yscale('log')
ax1.legend(fontsize=18);
if save_plots:
    plt.savefig('figs/C_l_WL.pdf')

In [ ]:
# GCphot angular power spectrum

fig1 = plt.figure()
ax1 = fig1.add_subplot(1, 1, 1)
ax1.tick_params(axis='both',which='both',direction='in')
for param in model.parameterization.sampled_params().keys():
        print('WARNING: param {} is being sampled! Possible mismatch between benchmark and theoretical prediction'.format(param))
pl_label = '$n_s = {:.2f}$'.format(model.parameterization.sampled_params()['ns'])
ax1 = pl_inst.plot_external_Cl_phot(bin_i, bin_j, ax1, probe='GC-Phot', pl_label='Benchmark')
ax1 = pl_inst.plot_Cl_phot(np.logspace(1, 3.6, 10), bin_i, bin_j, ax1, probe='GC-Phot', pl_colour='r', pl_linestyle='--', pl_label=pl_label)
ax1.set_xlabel(r'$\ell$', fontsize=20)
ax1.set_ylabel('$C_{1,1}^{\\rm{GCphot}}(\\ell) \\left[\\rm{sr}\\right]$')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.legend(fontsize=18);
if save_plots:
    plt.savefig('figs/C_l_GCphot.pdf')

In [ ]:
# Cross-correlation WL x GCphot angular power spectrum (GGL)

fig1 = plt.figure()
ax1 = fig1.add_subplot(1, 1, 1)
ax1.tick_params(axis='both',which='both',direction='in')
for param in model.parameterization.sampled_params().keys():
        print('WARNING: param {} is being sampled! Possible mismatch between benchmark and theoretical prediction'.format(param))
pl_label = '$n_s = {:.2f}$'.format(model.parameterization.sampled_params()['ns'])
ax1 = pl_inst.plot_external_Cl_XC(bin_i, bin_j, ax1, pl_label='Benchmark')
ax1 = pl_inst.plot_Cl_XC(np.logspace(1, 3.6, 10), bin_i, bin_j, ax1, pl_colour='r', pl_linestyle='--', pl_label=pl_label)
ax1.set_xlabel(r'$\ell$', fontsize=20)
ax1.set_ylabel('$C_{1,1}^{\\rm{GGL}}(\\ell) \\left[\\rm{sr}\\right]$')
ax1.set_xscale('log')
#ax1.set_yscale('log')
ax1.legend();
if save_plots:
    plt.savefig('figs/C_l_GGL.pdf')

In [ ]:
# Plotting the spectro observable: choose the multipole
multipole = 2

In [ ]:
# GCspectro observable

fig1 = plt.figure()
ax1 = fig1.add_subplot(1, 1, 1)
ax1.tick_params(axis='both',which='both',direction='in')
for param in model.parameterization.sampled_params().keys():
        print('WARNING: param {} is being sampled! Possible mismatch between benchmark and theoretical prediction'.format(param))
pl_label = '$n_s = {:.2f}$'.format(model.parameterization.sampled_params()['ns'])
ax1 = pl_inst.plot_external_GC_spectro("1.2", multipole, ax1, pl_label='Benchmark')
ax1 = pl_inst.plot_GC_spectro_multipole(1.2, np.linspace(0.01, 0.5), multipole, ax1, pl_colour='r', pl_linestyle='--', pl_label=pl_label)
ax1.set_xlabel(r'$k$ $\left[\rm{Mpc}^{-1}\right]$', fontsize=20)
ax1.set_ylabel(r'$P_{}$ $\left[\rm Mpc^3 \right]$'.format(multipole), fontsize=20)
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.legend();
if save_plots:
    plt.savefig('figs/P_l_k.pdf')